In [1]:
#Nuttavadee Autsavapanakit 62130500237

#import necessary library
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

# Loading the input data both train and test set (access through file path)
#Loading the train set 
train = pd.read_csv('../input/digit-recognizer/train.csv')
#Loading the test set
test = pd.read_csv('../input/digit-recognizer/test.csv')
submit = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [2]:
#Separate the data and the label into X_train and y_train
X_train = train.drop(['label'], 1).values
y_train = train['label'].values
x_test = test.values

In [3]:
X_train.shape

In [4]:
x_test.shape

In [5]:
#normalize the data image in order to work with the model
#converted value are in range 0 to 1
#divide the piixel value by 255
X_train = X_train/255
x_test = x_test/255

In [6]:
# perform categorical encoding for both the training and validation labels
y_train = keras.utils.to_categorical(y_train)

In [7]:
# Split the train and the validation set 
# test_size = 0.2 = test 20% : train 80%
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [8]:
# Importing Keras modules to build the midel with the stack of layers
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam
# Creating the model
#Instantiating the Model
def build_model(hp):
    model = keras.Sequential()
    #Creating the Input Layer
    model.add(Dense(units = 128, activation='relu', input_shape=(784,)))
    #Creating the Hidden Layer
    model.add(Dense(units = 128, activation='relu'))
    model.add(Flatten())
    # model.add(Dropout(0.2))
    model.add(Dropout(hp.Choice('rate', [0.2, 0.25, 0.3, 0.4, 0.5])))
    if hp.Boolean("dropout"):
        model.add(Dropout(hp.Choice('rate', [0.2, 0.25, 0.3, 0.4, 0.5])))
    #Creating the Output Layer - 10 category output
    model.add(Dense(10,activation = 'softmax'))

    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [10]:
import keras_tuner as kt

In [11]:
build_model(kt.HyperParameters())

In [12]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy', 
    max_trials=2) 

In [13]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
best_model = tuner.get_best_models()[0]

In [14]:
best_model.summary()

In [15]:
# #compile the model
# best_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# #training the model
# history = best_model.fit(
#     X_train, y_train, epochs=10, verbose=1, validation_data=(X_val, y_val)
# )

In [17]:
#use model to predict the x_test dataset (unknown dataset - not image in the training set)
pred = best_model.predict(x_test)
pred = np.argmax(pred, axis = 1)

In [18]:
submit['Label'] = pred
submit.to_csv("submission.csv", index=False)